In [42]:
# 1. csv파일을 load

# 2. csv파일의 '영엄장명' column을 읽어서 네이버 지도검색 url에서 넣어서 검색

# 3. 메뉴테그를 순서대로 읽은 뒤 대표메뉴 3가지와 가격을 추출 후 df에 저장
# column: 'menu_1_name, menu_1_price, menu_2_name, menu_2_price, menu_3_name, menu_3_price'

# 4 df를 csv파일로 저장

In [96]:
import os
from time import sleep
import tqdm

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [44]:
# CSV 파일 로드
df = pd.read_csv('../featureSelector/data/c4_general_cafeteria_chuncheon_CC_opened_SS-HJ_sampling.csv', encoding='utf-8')
mini_df = df.head(3)
print("CSV load complete")

# 결과를 저장할 DataFrame 초기화 column: 'index_cafeteria, cafeteria_name, menu_name, menu_price'
result_df = pd.DataFrame({
    'index_cafeteria': [-1],
    'cafeteria_name': [-1],
    'menu_name': [-1],
    'menu_price': [-1]
})

CSV load complete


In [94]:
def pagnSecond(driver, KEYWORD_INDEX, KEYWORD):
    sleep(1.5)
    button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
    button.click()
    sleep(1.5)
    menu_html = driver.page_source  # 페이지의 HTML 가져오기
    menu_html_soup = BeautifulSoup(menu_html, 'lxml')  # BeautifulSoup으로 파싱
    sleep(1.5)

    # DataFrame 초기화
    extracted_df = pd.DataFrame({
        'index_cafeteria': [KEYWORD_INDEX] * 3,
        'cafeteria_name': [KEYWORD] * 3
    })

    # 'name_menu' 클래스를 가진 모든 요소 찾기
    name_menus = menu_html_soup.find_all(class_='name_menu', limit=3)
    data_name = [menu.get_text() for menu in name_menus]

    # 'price_menu' 클래스를 가진 모든 요소 찾기
    price_menus = menu_html_soup.find_all(class_='price_menu', limit=3)
    data_price = [menu.get_text().replace('가격: ', '').replace('\n', '') for menu in price_menus]

    # 데이터 길이 확인 및 DataFrame 업데이트
    if len(data_name) == len(extracted_df):
        extracted_df['menu_name'] = data_name
        print("상위 3개 메뉴 추출 완료")
    else:
        print("메뉴 데이터의 길이가 맞지 않아 추가하지 않습니다.")

    if len(data_price) == len(extracted_df):
        extracted_df['menu_price'] = data_price
        print("메뉴별 가격 추출 완료")
    else:
        print("가격 데이터의 길이가 맞지 않아 추가하지 않습니다.")
        
    driver.quit()  # 웹 드라이버 종료

In [95]:
# 테스트용 변수 설정
i = 32
print("index 변수 설정 완료 i = ", i)
KEYWORD_INDEX = df['번호'][i]
KEYWORD = df['사업장명'][i]
URL = "https://m.map.kakao.com/actions/searchView?q=" + "춘천 " + KEYWORD

driver = webdriver.Chrome()  # 웹 드라이버 생성
driver.get(URL)  # 링크 열기
sleep(1.5)
serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱

try:
    pagnSecond(driver, KEYWORD_INDEX, KEYWORD)
except NoSuchElementException:
    print("해당 요소를 찾을 수 없습니다: .tit_g")
    # 필요한 경우 추가적인 오류 처리를 여기에 작성합니다.
    driver.quit()  # 웹 드라이버 종료


index 변수 설정 완료 i =  32


NameError: name 'NoSuchElementException' is not defined

In [69]:
extracted_df

,index_cafeteria,cafeteria_name,menu_name
0,7631,착한복돼지,생삼겹살
1,7631,착한복돼지,항정살
2,7631,착한복돼지,된장찌개


In [73]:
# result_df에 extracted_df의 1~3행을 추가
result_df = pd.concat([result_df, extracted_df])

In [74]:
result_df

,index_cafeteria,cafeteria_name,menu_name,menu_price
0,-1,-1,-1,-1
0,7607,리네,규돈,"가격: 6,500\n"
1,7607,리네,부타돈,"가격: 6,500\n"
2,7607,리네,신기돈,"가격: 7,000\n"
0,7609,한아름밥상,제육쌈밥 정식,"가격: 11,000\n"
1,7609,한아름밥상,자반 고등어 구이,"가격: 11,000\n"
2,7609,한아름밥상,제육백반,"가격: 7,000\n"
0,7610,60주년기념관카페마운틴,에스프레소,NaN
1,7610,60주년기념관카페마운틴,아메리카노,NaN
2,7610,60주년기념관카페마운틴,더치커피,NaN
